<a href="https://colab.research.google.com/github/muhibuddin12/tensorFlow/blob/master/Tugas_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle


# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, mode='rt', encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# split a loaded document into sentences
def to_pairs(doc):
	lines = doc.strip().split('\n')
	pairs = [line.split('\t') for line in  lines]
	return pairs

# clean a list of lines
def clean_pairs(lines):
	cleaned = list()
	# prepare regex for char filtering
	re_print = re.compile('[^%s]' % re.escape(string.printable))
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for pair in lines:
		clean_pair = list()
		for line in pair:
			# normalize unicode characters
			line = normalize('NFD', line).encode('ascii', 'ignore')
			line = line.decode('UTF-8')
			# tokenize on white space
			line = line.split()
			# convert to lowercase
			line = [word.lower() for word in line]
			# remove punctuation from each token
			line = [word.translate(table) for word in line]
			# remove non-printable chars form each token
			line = [re_print.sub('', w) for w in line]
			# remove tokens with numbers in them
			line = [word for word in line if word.isalpha()]
			# store as string
			clean_pair.append(' '.join(line))
		cleaned.append(clean_pair)
	return array(cleaned)

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
filename = 'corpus_15_baru.txt'
doc = load_doc(filename)
# split into indonesia-minang pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
# save clean pairs to file
save_clean_data(clean_pairs, 'indo-minang.pkl')
# spot check
for i in range(100):
  print('[%s] => [%s]' % (clean_pairs[i,0], clean_pairs[i,1]))

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
	dump(sentences, open(filename, 'wb'))
	print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('indo-minang.pkl')

# reduce dataset size
n_sentences = 10000
dataset = raw_dataset[:n_sentences, :]
# random shuffle
shuffle(dataset)
# split into train/test
train, test = dataset[1:9000], dataset[9000:]
# save
save_clean_data(dataset, 'indo-minang-both.pkl')
save_clean_data(train, 'indo-minang-train.pkl')
save_clean_data(test, 'indo-minang-test.pkl')


Saved: indo-minang.pkl
[keistimewaan tersebut dapat diketahui dari keaslian karangan dan bahasa hamka] => [kaistimewaan tu dapek dikatahui dari kaaslian karangan jo bahaso hamka]
[cherrybelle beat indonesia adalah salah satu konser tur keliling pertama grup musik asal indonesia cherrybelle] => [cherrybelle beat indonesia adolah salah satu konser tur kaliliang patamo grup musik asa indonesia cherrybelle]
[makamnya diziarahi oleh ribuan pengagumnya dari masa ke masa] => [makamnyo diziarahi dek ribuan pangagumnyo dari maso ka maso]
[talmaciu adalah sebuah kota yang terletak di county sibiu rumania tengah] => [talmaciu adolah sabuah kota nan ado di county sibiu rumania]
[akhir dari bulan ramadan dirayakan dengan sukacita oleh seluruh muslim di seluruh dunia] => [akhia dari bulan ramadan dirayoan jo sanang ati dek saluruah muslim di saluruah dunia]
[adat perpatih merupakan salah satu bentuk sistem demokrasi melayu] => [adaik parpatiah marupoan salah satu bantuak sistem demokrasi malayu]
[st

In [8]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint


# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y

# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model

# load datasets
dataset = load_clean_sentences('indo-minang-both.pkl')
train = load_clean_sentences('indo-minang-train.pkl')
test = load_clean_sentences('indo-minang-test.pkl')

# prepare indonesia tokenizer
ind_tokenizer = create_tokenizer(dataset[:, 0])
ind_vocab_size = len(ind_tokenizer.word_index) + 1
ind_length1 = max_length(dataset[:, 0])
ind_length = 9
print('Indo Vocabulary Size: %d' % ind_vocab_size)
print('Indo Max Length: %d' % (ind_length1))
# prepare minang tokenizer
min_tokenizer = create_tokenizer(dataset[:, 1])
min_vocab_size = len(min_tokenizer.word_index) + 1
min_length1 = max_length(dataset[:, 1])
min_length = 9
print('Minang Vocabulary Size: %d' % min_vocab_size)
print('Minang Max Length: %d' % (min_length1))



Indo Vocabulary Size: 13456
Indo Max Length: 21
Minang Vocabulary Size: 15624
Minang Max Length: 15


In [9]:
# prepare training data
trainX = encode_sequences(min_tokenizer, min_length, train[:, 1])
trainY = encode_sequences(ind_tokenizer, ind_length, train[:, 0])
trainY = encode_output(trainY, ind_vocab_size)

In [10]:
# prepare validation data
testX = encode_sequences(min_tokenizer, min_length, test[:, 1])
testY = encode_sequences(ind_tokenizer, ind_length, test[:, 0])
testY = encode_output(testY, ind_vocab_size)

In [11]:
# define model
model = define_model(min_vocab_size, ind_vocab_size, min_length, ind_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True)
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=5, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 9, 256)            3999744   
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 repeat_vector_1 (RepeatVect  (None, 9, 256)           0         
 or)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 9, 256)            525312    
                                                                 
 time_distributed_1 (TimeDis  (None, 9, 13456)         3458192   
 tributed)                                                       
                                                                 
Total params: 8,508,560
Trainable params: 8,508,560
No

In [12]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
	prediction = model.predict(source, verbose=0)[0]
	integers = [argmax(vector) for vector in prediction]
	target = list()
	for i in integers:
		word = word_for_id(i, tokenizer)
		if word is None:
			break
		target.append(word)
	return ' '.join(target)

# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
	actual, predicted = list(), list()
	for i, source in enumerate(sources):
		# translate encoded source text
		source = source.reshape((1, source.shape[0]))
		translation = predict_sequence(model, ind_tokenizer, source)
		raw_target, raw_src = raw_dataset[i]
		if i < 10:
			print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
		actual.append([raw_target.split()])
		predicted.append(translation.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

# load datasets
dataset = load_clean_sentences('indo-minang-both.pkl')
train = load_clean_sentences('indo-minang-train.pkl')
test = load_clean_sentences('indo-minang-test.pkl')
# prepare indonesia tokenizer
ind_tokenizer = create_tokenizer(dataset[:, 0])
ind_vocab_size = len(ind_tokenizer.word_index) + 1
#ind_length = max_length(dataset[:, 0])
ind_length = 9
# prepare minang tokenizer
min_tokenizer = create_tokenizer(dataset[:, 1])
min_vocab_size = len(min_tokenizer.word_index) + 1
#min_length = max_length(dataset[:, 1])
min_length = 9
# prepare data
trainX = encode_sequences(min_tokenizer, min_length, train[:, 1])
testX = encode_sequences(min_tokenizer, min_length, test[:, 1])

# load model
model = load_model('model.h5')
# test on some training sequences
print('train')
evaluate_model(model, ind_tokenizer, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, ind_tokenizer, testX, test)

OSError: ignored